# Day 9: Smoke Basin

[https://adventofcode.com/2021/day/9](https://adventofcode.com/2021/day/9)

## Description

### Part One

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the _low points_ - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are _four_ low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The _risk level_ of a low point is _1 plus its height_. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore _`15`_.

Find all of the low points on your heightmap. _What is the sum of the risk levels of all low points on your heightmap?_

### Part Two

Next, you need to find the largest basins so you know what areas are most important to avoid.

A _basin_ is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The _size_ of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The top-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The middle basin, size `14`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The bottom-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = 1134`.

_What do you get if you multiply together the sizes of the three largest basins?_

In [1]:
import numpy as np
import copy
from time import perf_counter

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

part1:

In [3]:
x = np.array([np.array(list(line),dtype=np.int8) for line in rawstring.splitlines()],dtype=np.int8)

In [4]:
def part1(x):
    return sum([view[1,1]+1
        for view in 
            np.lib.stride_tricks.sliding_window_view(
                    np.pad(x,pad_width=1, mode='constant', constant_values = 10), (3,3)
                ).reshape((-1,3,3))
        if np.all(view[1,1] < view[[0,2,1,1],[1,1,0,2]])
        ])
part1(x)

539

part2:
locate basins

In [5]:
def locLows(x): 
    padded_x = np.lib.stride_tricks.sliding_window_view(
        np.pad(x,pad_width=1, mode='constant', constant_values = 10)
        , (3,3))
    x_dim = padded_x.shape[0]
    y_dim = padded_x.shape[1]

    lows = []

    for i in range(x_dim):
        for j in range(y_dim):
            view = padded_x[i,j,:]
            if np.all(view[1,1] < view[[0,2,1,1],[1,1,0,2]]):
                lows.append((i+1,j+1))
    return lows

In [6]:
def part2(x):
    lows = locLows(x)
    basins = [[] for i in range(len(lows))]
    padded_x = np.pad(x,pad_width=1, mode='constant', constant_values = 10)
    for idx, coord in enumerate(lows):
        basins[idx].append(coord)
        for i,j in basins[idx]:
            for k,l in [(i-1,j),(i+1,j),(i,j+1),(i,j-1)]:
                if padded_x[k,l] > padded_x[i,j] and padded_x[k,l] < 9 and (not (k,l) in basins[idx]):
                    basins[idx].append((k,l))
        
    return np.product(sorted([len(b) for b in basins],reverse=True)[0:3])

part2(x)

736920